# This Notebook will run reliably.

In [ ]:
import requests
import classes
import json
import pprint
import pandas as pd
from datetime import date, datetime, timedelta
from openpyxl import load_workbook
import time
import os
import dateutil.relativedelta as rd

In [ ]:
# Initialize 
data = classes.pith()

In [ ]:
print((data.default_start,data.default_end))

In [ ]:
# Call Workamajig for employee data
response = requests.get(data.emp_api,headers=data.headers)

In [ ]:
print(response.reason)

In [ ]:
# Parse API payload into JSON
payload = json.loads(response.content)

In [ ]:
# Produce a list of employees and their identity keys
employees = []
for i in payload['data']['employee']:
    employees.append(i['employeeKey'])

In [ ]:
print(employees)

In [ ]:
# For every employee, retrieve their timesheets
sheets = pd.DataFrame()
for j in employees:
    api_url = data.time_api.format(j)
    response = requests.get(api_url,headers=data.headers)
    print((response.reason, response.status_code, response.elapsed, j ))
    payload = json.loads(response.content)
    for k in payload['data']['timesheet']:
        sheet = pd.DataFrame(k)
        sheets = pd.concat([sheets, sheet])
# Former employees will appear as 'Bad Requests'

In [ ]:
# Pull the time entries out of the timesheets
times = pd.DataFrame()
for k in sheets['TimeEntries']:
    df = pd.DataFrame(k,index=[0])
    times = pd.concat([times,df])

In [ ]:
# The username isn't on the time entry JSON, so we rejoin the data from sheets to name them.
names = sheets[['TimeSheetKey','UserName']].drop_duplicates()

In [ ]:
# Base table is the timesheets and time sessions of all employees since the inception of Workamajig
base = pd.merge(times,names,how="left",left_on="timeSheetKey",right_on="TimeSheetKey")

In [ ]:
# Convert the time entries into tasks so that you can figure out the estimates.
tasks = base.groupby(["projectFullName",
                      "projectNumber",
                      "taskID",
                      "workDate",
                      "UserName",
                      "serviceCode",
                      "serviceDescription",
                      "taskName", 
                      "campaignName",
                      "clientName",
                      "taskKey",
                      ],as_index=False).agg({"actualHours":"sum"})

In [ ]:
# Create Estimates
import numpy as np
estimates = tasks.groupby(['UserName','taskName','clientName'],as_index=False).agg({'actualHours':"mean"})
estimates = estimates.rename(columns={"actualHours":"estimate"})

In [ ]:
def api_cust():
    x = str(datetime.now())
    # makes the datetime a record in the operating system environment variables
    os.environ['API_attempt_at_'+x] = "1"
    ls = []
    # Looks through the variables for...
    for i in dict(os.environ).keys():
        # our date records...
        if i.__contains__('API_ATTEMPT'):
            #and turns them back into dates...
            x = datetime.strptime(i[15:],"%Y-%m-%d %H:%M:%S.%f")
            # )But not before making a list of variables that are just sitting out there)
            ls.append(i)
            # ...Checks to see if the date of the env variable record is older than 15 mins and then drops it if it is.
            if x < (datetime.now() - rd.relativedelta(minutes = 15)):
                del os.environ[i]
    # Depending on how many env variables exist there are so many tries left.
    if len(ls) == 0:
        m = "Five calls remaining."
    elif len(ls) == 1:
        m = "Four calls remaining."
    elif len(ls) == 2:
        m = "Three calls remaining."
    elif len(ls) == 3:
        m = "Two calls remaining."
    elif len(ls) == 4:
        m = "One call remaining."
    else:
        # prints some information about when you can try again.
        m = "Hey, you've overloaded the API. Ease off until "+ str(datetime.strptime(min(ls)[26:],"%H:%M:%S.%f") + rd.relativedelta(minutes = 15))[12:]
    print(m)
# Create list of all projects
def cust_report(data = None ) -> dict:
    api_cust()
    response = requests.get(data.cust_report_str,headers = data.headers)
    content = json.loads(response.content)
    proj_table = pd.DataFrame(content['data']['report'])
    cust_report_response = response
    project_table = proj_table
    return {"response":cust_report_response,"project_table":project_table}
# Call function
cr = cust_report(data=data)
# Project number list creator
def proj_numbers(cust_report: pd.DataFrame,data) -> list:
    # Table of all projects
    proj_list = cust_report
    # Date needs to be in Datetsime format
    proj_list['first_Task_Start_Date'] = pd.to_datetime(proj_list['first_Task_Start_Date'])
    # Project list of just the last year
    proj_list = proj_list[(proj_list['first_Task_Start_Date']>(data.timestamp - timedelta(days=data.history_in_days)))]
    # Only project number
    proj_list = proj_list[['project_Number']]
    # Return a copy
    proj_list = proj_list['project_Number'].tolist()
    return proj_list
# Call Function
pl = proj_numbers(cust_report=cr['project_table'],data=data)


In [ ]:
# Collect tasks for all projects
def get_tasks(projects: list, data, sleeper: float = 2.8) -> dict:
    x: int = 1
    payloads = {}
    task_collection: pd.DataFrame = pd.DataFrame()
    for i in projects:
        print(f"Now collecting data for project {i}, {x} of {len(projects)}...")
        x = x +1 
        response = requests.get(r"https://app6.workamajig.com/api/beta1/projects?id={}&includeTasks=true".format(i), headers = data.headers)
        content = json.loads(response.content)
        contentdatatasks = content['data']['tasks']
        tasks = pd.DataFrame(contentdatatasks)
        responseB = requests.get(r"https://app6.workamajig.com/api/beta1/tasks?projectNumber={}".format(i), headers = data.headers)
        contentB = json.loads(responseB.content)
        contentdatatasksB = contentB['data']['task']
        tasksB = pd.DataFrame(contentdatatasksB)
        complete_data = tasksB[["taskKey","actComplete"]]
        tasks = pd.merge(tasks,complete_data,how='left',on='taskKey')
        tasks = tasks.assign(project_Number = i)
        userlist: list = []
        for j in contentdatatasks:
            user = j['taskUser']
            for jj in user:
                userlist.append(jj)
        if userlist != 0:
            try:
                tasks = pd.merge(tasks, pd.DataFrame(userlist), how='left', on="taskKey")
            except Exception as e:
                print(e,end="...")
        task_collection = pd.concat([task_collection,tasks])
        pl = {"payload_for_"+str(i):response}
        payloads.update(pl)
        time.sleep(sleeper)
    return {"dataset":task_collection, "payloads":payloads}
tc = get_tasks(projects=pl,data=data)

In [ ]:
# Apply estimates to full list of tasks
pl = pd.DataFrame(pl,columns=["projectNumber"])
pl['projectNumber'] = pl['projectNumber'].astype('str')
taskset = tc['dataset']
taskset['projectNumber'] = taskset['projectNumber'].astype('str')
taskset['projectNumber'] = taskset['projectNumber'].str.replace(".0","")

master = pd.merge(pl,taskset,how='left',on='projectNumber')
estimates = estimates.rename(columns={'UserName':"userName"})
estimates = estimates.groupby(['userName','taskName'],as_index=False).agg({'estimate':np.mean})
master = pd.merge(master, estimates,how='left',on=["userName","taskName"])
master['actStart'] = pd.to_datetime(master['actStart']).dt.tz_localize(None)
master['actComplete'] = pd.to_datetime(master['actComplete']).dt.tz_localize(None)
master['planStart'] = pd.to_datetime(master['planStart']).dt.tz_localize(None)
master['planComplete'] = pd.to_datetime(master['planComplete']).dt.tz_localize(None)
master = master[master['taskName'].str.contains('[Cc}]lient')!=True]

unassigned = master[master['userName'].isna()==True]
unassigned = master[(master['planStart']>=data.default_start)&(master['planStart']>=data.default_end)]
unassigned = unassigned[['projectNumber']].drop_duplicates()

In [ ]:
# Print full dataset as DATA to the capacity planner
def summary_table(dataset: pd.DataFrame,start: date,end: date,dest_dir:str):
    # Then, overwrite the data tab.
    def write_excel(filename,sheetname,dataframe):
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
            try:
                writer.book.remove(writer.book[sheetname])
            except:
                print("Worksheet does not exist")
            finally:
                dataframe.to_excel(writer, sheet_name=sheetname,index=False)
                #writer.save()
    write_excel(dest_dir+r"\capacityplanner2.xlsx","DATA",dataset)
    write_excel(dest_dir+r"\capacityplanner2.xlsx","UNASSIGNED",unassigned)
    # Write variables into the spreadsheet
    wb = load_workbook(dest_dir+r"\capacityplanner2.xlsx",read_only=False)
    ws = wb["VARIABLES"]
    ws["A1"] = str("Variable")
    ws["A2"] = str("Start")
    ws["B2"] = str(start)
    ws["A3"] = str("End")
    ws["B3"] = str(end)
    ws["A4"] = str("Period Hours")

    if start.strftime("%d") =='01':
        b = "Early {} {}".format(start.strftime("%B"),start.strftime("%Y"))
    elif start.strftime("%d") == '15':
        b = "Late {} {}".format(start.strftime("%B"),start.strftime("%Y"))
    else:
        b = "On {}".format(start.strftime("%d%B%Y"))

    wb.save(dest_dir+r"\planners\planner {}.xlsx".format(b))
    wb.save(dest_dir+r"\capacityplanner2.xlsx")

summary_table(dataset=master, start=data.default_start, end=data.default_end,dest_dir=data.dest_dir)

In [ ]:
# Package as an application

In [ ]:
# Stand up on the server for the website